<a href="https://colab.research.google.com/github/Yashas-naidu/Adobe_Gensolve_Round_2/blob/main/adobe_gensolve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reading csv files

In [1]:
import numpy as np
def read_csv ( csv_path ):
  np_path_XYs = np . genfromtxt ( csv_path , delimiter = ',')
  path_XYs = []
  for i in np . unique ( np_path_XYs [: , 0]):
    npXYs = np_path_XYs [ np_path_XYs [: , 0] == i ][: , 1:]
    XYs = []
    for j in np . unique ( npXYs [: , 0]):
      XY = npXYs [ npXYs [: , 0] == j ][: , 1:]
      XYs . append ( XY )
    path_XYs . append ( XYs )
  return path_XYs

# Plotting the points

In [2]:
import matplotlib.pyplot as plt
def plot(path_XYs):
    fig, ax = plt.subplots(tight_layout=True, figsize=(5, 5))
    my_colours = ['red', 'blue', 'green']  # Define your colors here
    for i, XYs in enumerate(path_XYs):
        c = my_colours[i % len(my_colours)]
        for XY in XYs:
            ax.plot(XY[:, 0], XY[:, 1], c=c, linewidth=2)
        ax.set_aspect('equal')
    # Add any other necessary plot customization
    plt.show()



# Identifying shapes
- straight line
- circle
- ellipse
- rectangle and rounded rectangle
- regular polygon
- star shaped

In [3]:
#straight line
def is_straight_line(XY):
    # Check if all points lie on a straight line
    if len(XY) < 3:
        return True  # A line with two points is always straight
    x, y = XY[:, 0], XY[:, 1]
    return np.all(np.isclose(np.diff(y) / np.diff(x), (y[1] - y[0]) / (x[1] - x[0])))

#circle
def fit_circle(XY):
    x = XY[:, 0]
    y = XY[:, 1]
    A = np.column_stack((x, y, np.ones_like(x)))
    f = x**2 + y**2
    C, _, _, _ = np.linalg.lstsq(A, f, rcond=None)
    xc, yc, r2 = C[0]/2, C[1]/2, C[2]
    return xc, yc, np.sqrt(r2 + xc**2 + yc**2)

def is_circle(XY, tolerance=1e-2):
    xc, yc, r = fit_circle(XY)
    distances = np.sqrt((XY[:, 0] - xc)**2 + (XY[:, 1] - yc)**2)
    return np.all(np.abs(distances - r) < tolerance)

#ellipse
from scipy.optimize import least_squares

def fit_ellipse(XY):
    def ellipse_residuals(params, x, y):
        xc, yc, a, b, theta = params
        cos_theta, sin_theta = np.cos(theta), np.sin(theta)
        x_rot = cos_theta * (x - xc) + sin_theta * (y - yc)
        y_rot = -sin_theta * (x - xc) + cos_theta * (y - yc)
        return (x_rot / a)**2 + (y_rot / b)**2 - 1
    
    x = XY[:, 0]
    y = XY[:, 1]
    xc, yc = np.mean(x), np.mean(y)
    a, b = np.ptp(x) / 2, np.ptp(y) / 2
    theta = 0
    params_initial = [xc, yc, a, b, theta]
    result = least_squares(ellipse_residuals, params_initial, args=(x, y))
    return result.x

#rectangle
def is_rectangle(XY, tolerance=1e-2):
    if len(XY) != 4:
        return False
    distances = np.sqrt(np.sum(np.diff(XY, axis=0)**2, axis=1))
    distances = np.append(distances, np.sqrt(np.sum((XY[0] - XY[-1])**2)))
    opposite_sides_equal = np.isclose(distances[0], distances[2], atol=tolerance) and np.isclose(distances[1], distances[3], atol=tolerance)
    diagonals_equal = np.isclose(np.sqrt(np.sum((XY[0] - XY[2])**2)), np.sqrt(np.sum((XY[1] - XY[3])**2)), atol=tolerance)
    return opposite_sides_equal and diagonals_equal

#rounded rectangle
def is_rounded_rectangle(XY, tolerance=1e-2):
    if len(XY) < 8:
        return False
    # Simplified approach: look for rounded corners by checking curvatures at vertices
    from scipy.spatial.distance import cdist
    distances = cdist(XY, XY)
    num_close_points = np.sum(distances < tolerance, axis=0)
    return np.all(num_close_points >= 3)

#Regular polygon
def is_regular_polygon(XY, tolerance=1e-2):
    n = len(XY)
    if n < 3:
        return False
    side_lengths = np.sqrt(np.sum(np.diff(XY, axis=0)**2, axis=1))
    side_lengths = np.append(side_lengths, np.sqrt(np.sum((XY[0] - XY[-1])**2)))
    mean_side_length = np.mean(side_lengths)
    return np.all(np.abs(side_lengths - mean_side_length) < tolerance)

#star shape
def is_star_shape(XY, tolerance=1e-2):
    # Check for multiple radial arms from a central point
    from scipy.spatial.distance import cdist
    distances = cdist(XY, XY)
    central_point = np.mean(XY, axis=0)
    radial_distances = np.sqrt(np.sum((XY - central_point)**2, axis=1))
    peaks, _ = find_peaks(radial_distances, height=np.mean(radial_distances))
    return len(peaks) > 2  # A star shape should have more than 2 arms

def find_peaks(data, height=None):
    import scipy.signal
    peaks, properties = scipy.signal.find_peaks(data, height=height)
    return peaks, properties




# completing the curves and check for symmetry


In [5]:
import cv2
import numpy as np
from scipy.interpolate import splprep, splev

def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, binary = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)
    return binary

def find_contours(binary_image):
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def interpolate_curve(contour, gap_threshold=10):
    new_contour = []
    for i in range(len(contour) - 1):
        pt1, pt2 = contour[i][0], contour[i + 1][0]
        new_contour.append(pt1)
        if np.linalg.norm(pt1 - pt2) > gap_threshold:
            num_points = int(np.linalg.norm(pt1 - pt2) // gap_threshold)
            for j in range(1, num_points):
                new_point = pt1 + j * (pt2 - pt1) / num_points
                new_contour.append(new_point.astype(int))
    new_contour.append(contour[-1][0])
    return np.array(new_contour).reshape((-1, 1, 2))

def smooth_curve(contour, s=0.5):
    x, y = contour[:, 0, 0], contour[:, 0, 1]
    tck, u = splprep([x, y], s=s)
    new_points = splev(u, tck)
    new_contour = np.vstack(new_points).T.astype(np.int32).reshape(-1, 1, 2)
    return new_contour

def visualize_curves(image_path, contours, completed_contours):
    img = cv2.imread(image_path)
    cv2.drawContours(img, contours, -1, (0, 255, 0), 2)
    cv2.drawContours(img, completed_contours, -1, (255, 0, 0), 2)
    cv2.imshow("Curves", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def complete_curves(image_path):
    binary_image = preprocess_image(image_path)
    contours = find_contours(binary_image)
    
    completed_contours = []
    for contour in contours:
        completed_contour = interpolate_curve(contour)
        smoothed_contour = smooth_curve(completed_contour)
        completed_contours.append(smoothed_contour)
    
    visualize_curves(image_path, contours, completed_contours)

def check_symmetry(contour, axis='x'):
    points = contour[:, 0, :]
    if axis == 'x':
        mid_x = np.mean(points[:, 0])
        reflected_points = points.copy()
        reflected_points[:, 0] = 2 * mid_x - points[:, 0]
    else:
        mid_y = np.mean(points[:, 1])
        reflected_points = points.copy()
        reflected_points[:, 1] = 2 * mid_y - points[:, 1]
    
    distances = cdist(points, reflected_points)
    min_distances = np.min(distances, axis=1)
    return np.mean(min_distances)

def find_symmetry(contour):
    x_symmetry = check_symmetry(contour, 'x')
    y_symmetry = check_symmetry(contour, 'y')
    if x_symmetry < y_symmetry:
        return 'x', x_symmetry
    else:
        return 'y', y_symmetry

def fit_bezier_curve(points):
    n = len(points)
    t = np.linspace(0, 1, n)
    A = np.array([t**2, t, np.ones_like(t)]).T
    bx, _, _, _ = np.linalg.lstsq(A, points[:, 0], rcond=None)
    by, _, _, _ = np.linalg.lstsq(A, points[:, 1], rcond=None)
    return bx, by

def bezier_points(bx, by, n_points=100):
    t = np.linspace(0, 1, n_points)
    curve = np.array([bx[0]*t**2 + bx[1]*t + bx[2], by[0]*t**2 + by[1]*t + by[2]]).T
    return curve

def visualize_curves(image_path, contours, symmetries):
    img = cv2.imread(image_path)
    for contour, symmetry in zip(contours, symmetries):
        cv2.drawContours(img, [contour], -1, (0, 255, 0), 2)
        axis, _ = symmetry
        if axis == 'x':
            cv2.line(img, (int(np.mean(contour[:, 0, 0])), 0), (int(np.mean(contour[:, 0, 0])), img.shape[0]), (255, 0, 0), 2)
        else:
            cv2.line(img, (0, int(np.mean(contour[:, 0, 1]))), (img.shape[1], int(np.mean(contour[:, 0, 1]))), (255, 0, 0), 2)
    cv2.imshow("Symmetries", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def identify_and_fit_curves(image_path):
    binary_image = preprocess_image(image_path)
    contours = find_contours(binary_image)
    
    symmetries = []
    for contour in contours:
        symmetry = find_symmetry(contour)
        symmetries.append(symmetry)
    
    visualize_curves(image_path, contours, symmetries)

# Example Usage
image_path = 'path/to/your/image.png'
identify_and_fit_curves(image_path)


# Comparing the results

In [ ]:
# Read and plot an example CSV file
csv_file_path = r'C:\Users\YASHAS\Downloads\Adobe_Gensolve_Round_2\problems\problems\frag0.csv'
paths_XYs = read_csv(csv_file_path)
plot(paths_XYs)

# Check if a path is a straight line
for path in paths_XYs:
    for XY in path:
        if is_straight_line(XY):
            print("Straight line detected")

# Check if a path is a circle
for path in paths_XYs:
    for XY in path:
        if is_circle(XY):
            print("Circle detected")

for path in paths_XYs:
    for XY in path:
        if is_rectangle(XY):
            print("Rectangle detected")
        elif is_rounded_rectangle(XY):
            print("Rounded rectangle detected")

# Check if a path is a regular polygon
for path in paths_XYs:
    for XY in path:
        if is_regular_polygon(XY):
            print("Regular polygon detected")

# Check if a path is a star shape
for path in paths_XYs:
    for XY in path:
        if is_star_shape(XY):
            print("Star shape detected")

# Check for reflection symmetry
for path in paths_XYs:
    for XY in path:
        if has_reflection_symmetry(XY):
            print("Reflection symmetry detected")


# Complete an incomplete curve
completed_path = complete_curve(paths_XYs[0][0])
plot([completed_path])

# Compare with expected solution
expected_csv_file_path = 'path/to/your/expected_solution.csv'
expected_paths_XYs = read_csv(expected_csv_file_path)
for path, expected_path in zip(paths_XYs, expected_paths_XYs):
    for XY, expected_XY in zip(path, expected_path):
        if compare_results(XY, expected_XY):
            print("Match found")


In [17]:
def polylines2svg(paths_XYs, svg_path):
    W, H = 0, 0
    colours = ['red', 'blue', 'green']
    
    # Calculate dimensions
    for path_XYs in paths_XYs:
        for XY in path_XYs:
            W, H = max(W, np.max(XY[:, 0])), max(H, np.max(XY[:, 1]))
    
    padding = 0.1
    W, H = int(W + padding * W), int(H + padding * H)
    
    # Create a new SVG drawing
    dwg = svgwrite.Drawing(svg_path, profile='tiny', shape_rendering='crispEdges')
    group = dwg.g()
    
    for i, path in enumerate(paths_XYs):
        path_data = []
        c = colours[i % len(colours)]
        for XY in path:
            path_data.append(f"M {XY[0, 0]} {XY[0, 1]}")
            for j in range(1, len(XY)):
                path_data.append(f"L {XY[j, 0]} {XY[j, 1]}")
            if not np.allclose(XY[0], XY[-1]):
                path_data.append("Z")
        group.add(dwg.path(d=' '.join(path_data), fill=c, stroke='none', stroke_width=2))
    
    dwg.add(group)
    dwg.save()
    
    # Convert SVG to PNG
    png_path = svg_path.replace('.svg', '.png')
    fact = max(1, 1024 // min(H, W))
    cairosvg.svg2png(url=svg_path, write_to=png_path, parent_width=W, parent_height=H, output_width=fact * W, output_height=fact * H, background_color='white')
    
    return